In [1]:
HTML(read(open("style.css"), String))

HTML{String}("<link href='http://fonts.googleapis.com/css?family=Alegreya+Sans:100,300,400,500,700,800,900,100italic,300italic,400italic,500italic,700italic,800italic,900italic' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=Arvo:400,700,400italic' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=PT+Mono' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=Shadows+Into+Light' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=Philosopher:400,700,400italic,700italic' rel='stylesheet' type='text/css'>\r\n\r\n<style>\r\n\r\n@font-face {\r\n    font-family: \"Computer Modern\";\r\n    src: url('http://mirrors.ctan.org/fonts/cm-unicode/fonts/otf/cmunss.otf');\r\n}\r\n\r\n.container { width: 100% }\r\n\r\n/* Formatting for header cells */\r\n.text_cell_render h1 {\r\n    font-family: 'Philosopher', sans-serif;\r\n    font-weight: 400;\r\n    font-size: 2.2em;\r\n    line-height: 100%;\r\n    color: rgb(0, 80, 120);\r\n    margin-bottom: 0.1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\t\r\n.text_cell_render h2 {\r\n    font-family: 'Philosopher', serif;\r\n    font-weight: 400;\r\n    font-size: 1.9em;\r\n    line-height: 100%;\r\n    color: rgb(200,100,0);\r\n    margin-bottom: 0.1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\t\r\n\r\n.text_cell_render h3 {\r\n    font-family: 'Philosopher', serif;\r\n    margin-top:12px;\r\n    margin-bottom: 3px;\r\n    font-style: italic;\r\n    color: rgb(94,127,192);\r\n}\r\n\r\n.text_cell_render h4 {\r\n    font-family: 'Philosopher', serif;\r\n}\r\n\r\n.text_cell_render h5 {\r\n    font-family: 'Alegreya Sans', sans-serif;\r\n    font-weight: 300;\r\n    font-size: 16pt;\r\n    color: grey;\r\n    font-style: italic;\r\n    margin-bottom: .1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\r\n\r\n.text_cell_render h6 {\r\n    font-family: 'PT Mono', sans-serif;\r\n    font-weight: 300;\r\n    font-size: 10pt;\r\n    color: grey;\r\n    margin-bottom: 1px;\r\n    margin-top: 1px;\r\n}\r\n\r\n.text_cell_render em {\r\n    font-family: 'Philosopher', sans-serif;\r\n    color:        blue;\r\n    background-color: rgb(255,220,180);\r\n    font-size:    110%;\r\n    margin-left:   2px;\r\n    margin-right:  2px;\r\n    font-weight:   100;\r\n}\r\n\r\n.text_cell_render b {\r\n    color:            rgb(255,195,195);\r\n    background-color: rgb(0,0,0);\r\n    font-size:    110%;\r\n    margin-left:   2px;\r\n    margin-right:  2px;\r\n    font-weight:   650;\r\n}\r\n\r\n.text_cell_render tt {\r\n    font-size:    120%;\r\n    margin-left:   2px;\r\n    margin-right:  2px;\r\n    font-weight:   150;\r\n}\r\n\r\n.Codemirror {\r\n    font-family: \"PT Mono\";\r\n    font-size: 100%;\r\n}\r\n\r\n</style>\r\n\r\n")

# Advanced Board Struct

In [2]:
using Chess
using NBInclude

In [3]:
@nbinclude("Repetition.ipynb")

decrementHash! (generic function with 1 method)

This notebook contains the `Advanced Board` (short: `AdvBoard`) struct which bundles additional information about the board to the board. As the `::Board` Class of Chess.jl does not contain all the information we need to the board this will be expanded to contain the following additional contents:

1. `score::Int64` is the static evaluation of the board in centipawn
1. `hash::UInt64`  is the hash of the board
1. `repCounter::Dict{UInt64, Int8}` is a Dictionary that counts how often a position as occured

In [4]:
struct AdvBoard
    board::Board
    score::Int64
    hash::UInt64
    repCounter::Dict{UInt64, Int8}
end

## Constructor for AdvBoard

In [5]:
function AdvBoard(board::Board)::AdvBoard
    score = evaluate_position(board)
    hash  = zobrist_hash(board)
    repCounter = Dict{UInt64, Int8}()
    incrementHash!(repCounter, hash)
    return AdvBoard(board, score, hash, repCounter)
end

AdvBoard

In [6]:
function domoveAdv!(ABoard::AdvBoard, move::Move, score_opt=true)
    oldscore = ABoard.score
    oldhash = ABoard.hash
    if score_opt
        next_score = evaluate_move_Adv(ABoard, move)
        ABoard.score = next_score
    end
    next_hash = zobrist_hash_Adv(ABoard, move)
    undoinfo = domove!(ABoard.board, move)
    ABoard.hash  = next_hash
    incrementHash!(ABoard.repCounter, hash)
    return (oldscore, oldhash, undoinfo)
end

domoveAdv! (generic function with 2 methods)

In [7]:
function undomoveAdv!(ABoard::AdvBoard, undoinfo)
    decrementHash!(ABoard.repCounter, ABoard.hash)
    undomove!(ABoard.board, undoinfo[3])
    ABoard.score = undoinfo[1]
    ABoard.hash  = undoinfo[2]
end

undomoveAdv! (generic function with 1 method)